<a href="https://colab.research.google.com/github/froghop/ci2_hackathon/blob/main/ci2Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ci2 Hackathon

Data: https://drive.google.com/drive/folders/1-9lhZfnLHlUBwmCNyQQNN4Qfu4quUSrL

Zipped Data: https://github.com/froghop/ci2_hackathon/blob/main/test-20220623T221903Z-001.zip

Notes document: https://docs.google.com/document/d/1u5oVv_N4fzWZlmf9Rpj-SevUGZmRTjZ2_ufCgMUSycQ/edit?usp=sharing

# Data imports

In [1]:
# Import python packages
from skimage.transform import resize

from torch.functional import Tensor
import tensorflow as tf
import traceback
import sys

import tensorflow as tf
import tensorflow.keras

from tensorflow.keras import layers
from tensorflow.keras.applications.vgg16 import VGG16


import torch
import fastai.callback.all
from fastai import *
from fastai.vision import *
from fastai.callback import *
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
from torchvision.utils import make_grid
import torch.nn.functional as F

import copy
import matplotlib.pyplot as plt
import pickle
import numpy as np
import pandas as pd
import random
import tarfile
import time
import urllib
import os
import cv2
import sklearn

# Data Loader

Upload the zipped folder of the test data (download from github: https://github.com/froghop/ci2_hackathon/blob/main/test-20220623T221903Z-001.zip) into the files portion of this notebook. Then unzip the files.

In [2]:
!unzip /content/test-20220623T221903Z-001.zip
!unzip /content/train-20220624T140353Z-001.zip

Archive:  /content/test-20220623T221903Z-001.zip
  inflating: test/5020_T10.png       
  inflating: test/5001_T6.png        
  inflating: test/6014_T10.png       
  inflating: test/6014_L2.png        
  inflating: test/5049_T5.png        
  inflating: test/5040_L1.png        
  inflating: test/5049_T11.png       
  inflating: test/5040_T8.png        
  inflating: test/5001_T9.png        
  inflating: test/6014_T9.png        
  inflating: test/5049_T8.png        
  inflating: test/5049_L3.png        
  inflating: test/6014_L3.png        
  inflating: test/6014_T12.png       
  inflating: test/6014_T11.png       
  inflating: test/5040_T7.png        
  inflating: test/6014_T6.png        
  inflating: test/5001_T11.png       
  inflating: test/5049_T12.png       
  inflating: test/5001_T12.png       
  inflating: test/5020_T5.png        
  inflating: test/5049_T6.png        
  inflating: test/5020_L1.png        
  inflating: test/5020_T6.png        
  inflating: test/6014_L1.png        
 

In [2]:
root_dir = '/content/'
test_dir = '/content/test'
train_dir = '/content/train'

In [3]:
train_files = os.listdir(train_dir)
train_files.sort()

#the following code creates a list of filepaths of all images in our test data
train_files_filepaths = [os.path.join(train_dir, single_image) for single_image in train_files]
len(train_files_filepaths) # how many images we have to work with

1129

In [4]:
test_list_of_images = os.listdir(test_dir)
test_list_of_images.sort()

#the following code creates a list of filepaths of all images in our test data
test_list_of_images_filepaths = [os.path.join(test_dir, single_image) for single_image in test_list_of_images]
len(test_list_of_images) # how many images we have to work with

203

In [5]:
# verify there are only images in this dataset and no labels
for _ in test_list_of_images:
  if ".png" in _:
    pass
  else:
    print(_)

In [6]:
train_csv = ""
train_list_of_images_filepaths = []

In [7]:
for _ in train_files_filepaths:
  if ".png" in _:
    pass
  else:
    train_csv = os.path.join(train_dir, _)
    print("train_csv:", train_csv)
    train_files_filepaths.remove(_)

train_csv: /content/train/data.csv


In [8]:
train_csv = pd.read_csv(train_csv) #conver csv into pandas dataframe
train_csv.describe() #summarize the pandas dataframe

,Unnamed: 0,presence_of_fracture
count,1128.000000,1128.000000
mean,563.500000,0.091312
std,325.769857,0.288180
min,0.000000,0.000000
25%,281.750000,0.000000
50%,563.500000,0.000000
75%,845.250000,0.000000
max,1127.000000,1.000000


In [9]:
train_csv

,Unnamed: 0,img,presence_of_fracture
0,0,1001_L1.png,1.0
1,1,1001_L2.png,0.0
2,2,1001_L3.png,1.0
3,3,1001_T10.png,0.0
4,4,1001_T11.png,0.0
...,...,...,...
1123,1123,9006_T11.png,0.0
1124,1124,9006_T12.png,0.0
1125,1125,9006_T7.png,0.0
1126,1126,9006_T8.png,0.0


The following functions and code is to load the .png images into an array of pixels and put them in a dictionary with their filename. It also resizes the files to a desired file size for our machine learning model.

In [10]:
#from ci2 notebook pt 2

def standardize_image(img):
    '''Two methods of standardizing images are shown below, each with different
    advantages. Min-max scaling puts all features into the same scale.
    Z-scoring is better at dealing with outliers.'''

    #Z-scoring: zero-center and standardize by standard deviation
    #standardized_img = img - np.mean(img)
    #standardized_img /= np.std(standardized_img)

    #Min-max scaling: normalize to [0 1]
    standardized_img = (img - np.min(img))/(np.max(img) - np.min(img))

    return standardized_img

In [11]:
# DATA AUGMENTATION

# List of random image transformations for data augmentation
transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(10, fill=0.5),
                                transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), fill=0.5),
                                transforms.Normalize((0.485, 0.456, 0.406), 
                                                     (0.229, 0.224, 0.225))])
'''
augmented_images = []

for img in data_png:
  augmented_images += transform(img)

master_data = data_png + augmented_images
master_data.shuffle()

# Plot the original image
fig = plt.figure(figsize = (10, 5))
plt.subplot(1, 2, 1)
plt.imshow(original[:, :, :].numpy().transpose(1, 2, 0))
plt.clim([0,1])
plt.axis('off')
plt.title('Original: ' + label)

# Plot an example of the transformed image
plt.subplot(1,2,2)
tf_img = transformed[:, :, :].numpy().transpose(1, 2, 0)
# Purely for visualization purposes, scale pixel values to be from [0,1]. 
# Actual DL network would not do this last step (normalized output would be sufficient)
tf_img = (tf_img - np.min(tf_img))/(np.max(tf_img) - np.min(tf_img))
plt.imshow(tf_img)
plt.clim([0,1])
plt.axis('off')
plt.title('Transformed: ' + label)
'''

"\naugmented_images = []\n\nfor img in data_png:\n  augmented_images += transform(img)\n\nmaster_data = data_png + augmented_images\nmaster_data.shuffle()\n\n# Plot the original image\nfig = plt.figure(figsize = (10, 5))\nplt.subplot(1, 2, 1)\nplt.imshow(original[:, :, :].numpy().transpose(1, 2, 0))\nplt.clim([0,1])\nplt.axis('off')\nplt.title('Original: ' + label)\n\n# Plot an example of the transformed image\nplt.subplot(1,2,2)\ntf_img = transformed[:, :, :].numpy().transpose(1, 2, 0)\n# Purely for visualization purposes, scale pixel values to be from [0,1]. \n# Actual DL network would not do this last step (normalized output would be sufficient)\ntf_img = (tf_img - np.min(tf_img))/(np.max(tf_img) - np.min(tf_img))\nplt.imshow(tf_img)\nplt.clim([0,1])\nplt.axis('off')\nplt.title('Transformed: ' + label)\n"

In [12]:
# main data component
train_data_numpy = np.empty([0,224,224,3])

In [13]:
def match_load_and_resize(image_filepaths, csv, x, y, data_numpy=train_data_numpy):
  """
  Reads the images, resizes them, and saves them along with their pertinent 
  identifiers into dic. 
  @param path: the root path to begin file extraction from
  @param x: desired x-cord
  @param y: desired y-cord

  """
  data = []
  labels = []
  counter = 1
  for f in image_filepaths:
      if ".png" in f:
        imgpng = cv2.imread(f)
        imgpng_resized = resize(imgpng, (x,y))
        fn = f.split('/')[3] #filename to match to the csv dataframe to --> in format of "900_T6.png"
        try:
          label = train_csv.loc[train_csv['img'] == fn]['presence_of_fracture'].values[0]
          imgpng_resized = np.expand_dims(imgpng_resized, 0)
          imgpng_resized = standardize_image(imgpng_resized)
          #aug_img = transform(imgpng_resized)

          data_numpy = np.concatenate([data_numpy,imgpng_resized])
          #data_numpy = np.concatenate([data_numpy,aug_img])

          labels += label
          counter +=1
        except: 
          print("Error")
          print(traceback.format_exc())

  return labels

  print("Matching, Loading and resizing successful for " + str(counter) + " datapoints.")

In [14]:
labels = match_load_and_resize(train_files_filepaths, train_csv, 224, 224)

In [15]:
labels

array([], dtype=float64)

In [16]:
train_labels_tensor = tf.convert_to_tensor(labels)

In [17]:
train_labels_tensor

<tf.Tensor 'Const:0' shape=(0,) dtype=float64>

In [18]:
train_data_tensor = tf.convert_to_tensor(train_data_numpy)

In [19]:
test_data_numpy = np.empty([0,224,224,3])

In [20]:
def load_and_resize(test_image_filepaths, x, y, test_data_numpy=test_data_numpy):
  """
  Reads the images, resizes them, and saves them along with their pertinent 
  identifiers into dic. 
  @param path: the root path to begin file extraction from
  @param x: desired x-cord
  @param y: desired y-cord

  """
  data = []
  counter = 1
  for f in test_image_filepaths:
      if ".png" in f:
        imgpng = cv2.imread(f)
        imgpng_resized = resize(imgpng, (x,y))
        fn = f.split('/')[3] #filename to match to the csv dataframe to --> in format of "900_T6.png"
        try:
          imgpng_resized = np.expand_dims(imgpng_resized, 0)
          imgpng_resized = standardize_image(imgpng_resized)

          test_data_numpy = np.concatenate([test_data_numpy,imgpng_resized])

          counter +=1
        except: 
          print("Error")
          print(traceback.format_exc())   
  print("Loading and resizing successful for " + str(counter) + " images.")
  return data

In [21]:
#this will take a while
test_data = load_and_resize(test_list_of_images_filepaths, x=224, y=224)

Loading and resizing successful for 204 images.


In [22]:
test_data_tensor = tf.convert_to_tensor(test_data)

# Data Splitter

In [ ]:
# Subset of downloaded training set needs to be split into validation data
val_ratio = 0.2 # Split 20% of training data into validation

# Set random seed for reproducibility of validation split
random.seed(14)
torch.manual_seed(14)
np.random.seed(14)

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

test_data, train_data = sklearn.model_selection.train_test_split(data_png, test_size=0.2, train_size=0.8, 
                                         random_state=14, shuffle=True, stratify=None)

#A well designed network architecture

Resource: https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/

https://www.analyticsvidhya.com/blog/2021/06/transfer-learning-using-vgg16-in-pytorch/

Above resourse states that VGG net is one of better pre-trained models for unsupervised image classification, which is the problem we are tasked with because we have no labels and want our data to split into 3 categories.

In [41]:
!pip install tensorflow==1.15 --quiet
!pip install keras==2.2.4 --quiet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
from tensorflow.keras.applications.vgg16 import VGG16

In [33]:
# https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/

base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
                    include_top = False, # Leave out last FC layer
                    weights = 'imagenet')

In [34]:
for layer in base_model.layers:
  layer.trainable = False #want to freeze certain layers

In [46]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [35]:
base_model.output

<tf.Tensor 'block5_pool_5/MaxPool:0' shape=(?, 7, 7, 512) dtype=float32>

In [49]:
output = base_model.output

In [37]:
base_model.input

<tf.Tensor 'input_6:0' shape=(?, 224, 224, 3) dtype=float32>

In [51]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(output) #(base_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x) # keep as one for binary classification


AttributeError: ignored

In [52]:
# additional modifications???
batch_size=16
steps_per_epoch = 250/batch_size

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay( 
    0.001, 
    decay_steps=steps_per_epoch*1000, 
    decay_rate=1,
    staircase=False)

In [53]:
model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc']) #use binary crossentropy to help with the binary operation

NameError: ignored



---



---



#Alternate Model

ALTERNATE SOLUTION, BASED ON CI2 BOOTCAMP TUTORIAL INFORMATION

TODO: EDIT TO FIT OUR DATA AND SITUATION

In [ ]:
#Define function that reads and resizes images
def read_image(fname,crop_side_dim):
  img = Image.open(fname)
  #we want all of our images to be grayscale images (1 channel) and not RGB (3 channels)
  #in order for the pre-processing of the images later on to work properly!
  if len(np.array(img).shape) > 2:
    img = img.convert('L')
  img = np.array(img)
  img = scipy.ndimage.zoom(img,crop_side_dim/img.shape[0],order = 3)
  return img

# Form Dataset class that can extract images, corresponding labels of dataset
batch_size  = 7
num_workers = 1
class XRayData(Dataset):
    def __init__(self, labels_df, set_name,transforms = None,
        resized_image_length = 75):
        
        if set_name == 'train':
            self.labels_df = labels_df.loc[labels_df.Set == 0, :]
        elif set_name == 'val':
            self.labels_df = labels_df.loc[labels_df.Set == 1, :]
        elif set_name == 'test':
            self.labels_df = labels_df.loc[labels_df.Set == 2, :]
        else:
            print("Wrong set name was given")

        self.images     = list(self.labels_df['Image'])
        self.labels     = list(self.labels_df['Label'])
        self.len        = len(self.labels)
        self.transforms = transforms
        self.resized_image_length = resized_image_length

    def __getitem__(self, index):
        'Generates one sample of data'

        image_path = self.images[index]
        label      = self.labels[index]

        #read image
        image    = read_image(image_path,self.resized_image_length)

        #Normalize the image per exercise above
        image = standardize_image(image)

        #Convert image from numpy array to pytorch tensor
        image = np.stack([image,image,image],axis = 0) # 3 x resized_image_length x resized_image_length
        image = image.astype(np.float64)
        image = torch.from_numpy(image)

        #Do specified transforms, if parameter is set
        if self.transforms:
            image = self.transforms(image)

        label = torch.from_numpy(np.asarray(label))
        return image.type(torch.FloatTensor), label.type(torch.LongTensor)

    def __len__(self):
        return self.len

trainset = XRayData(labels_df=full_df, set_name='train')
valset   = XRayData(labels_df=full_df, set_name='val')
testset  = XRayData(labels_df=full_df, set_name='test')
print("Length of training set:", trainset.len)
print("Length of validation set:", valset.len)
print("Length of test set:", testset.len)

#Create dataloaders
trainset_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True,
                             num_workers=num_workers,
                             drop_last=True)

valset_loader = DataLoader(valset, batch_size=batch_size, shuffle=False,
                             num_workers=num_workers,
                             drop_last=False)

testset_loader = DataLoader(testset, batch_size=batch_size, shuffle=False,
                             num_workers=num_workers,
                             drop_last=False)

# A cool loss function

Using Binary Crossentropy

# A training loop

In [ ]:
#general structure of fitting model, based on resourse
vgghist = model.fit(x=train_data_tensor, y=train_labels_tensor, steps_per_epoch = steps_per_epoch, epochs = 1)

# A careful validation/evaluation strategy

In [ ]:
preds = vgghist.predict(test_data_tensor)

In [ ]:
np.savetxt('preds.csv', preds, delimiter=',')